# **Importamos las dependencias**

In [2]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

from google.cloud import storage
from typing import List, Dict, Any

import math

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [3]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

In [4]:
from vertexai.generative_models import GenerationConfig
from vertexai.generative_models import GenerationResponse

# **Configuración de las credenciales**

In [15]:
PROJECT_ID = 'ai-innovation-wivboost'                                           # ID del proyecto de Google Cloud
LOCATION = 'us-central1'                                                        # Región donde se encuentran los recursos de Vertex AI
BUCKET_NAME = 'vm-prueba'                                                       # Nombre del bucket de Google Cloud Storage donde se almacenan los videos
VIDEO_FOLDER_PATH = 'grabaciones-2-julio/partido-canal5/fragmentacion'          # Ruta del folder dentro del bucket donde se encuentran los videos

In [16]:
# Iniciamos Vertex AI con el proyecto y la ubicación especificados.
init(project = PROJECT_ID, location = LOCATION)

# **Funciones**

In [7]:
def obtener_nombres_de_archivos(nombre_bucket: str, nombre_carpeta: str) -> List[str]:
    """
    Obtiene una lista con los nombres de los archivos de una "carpeta" en un bucket.

    Args:
        nombre_bucket: El nombre del bucket de Cloud Storage.
        nombre_carpeta: La ruta de la carpeta (prefijo) dentro del bucket.

    Returns:
        Una lista de strings con los nombres de los archivos.
        Retorna una lista vacía si la carpeta no existe o hay un error.
    """
    try:
        cliente_storage = storage.Client()
        nombres_archivos = []

        # Aseguramos que el prefijo termine con '/' para que actúe como una carpeta
        if nombre_carpeta and not nombre_carpeta.endswith('/'):
            nombre_carpeta += '/'

        # Usamos el parámetro 'prefix' para filtrar por la carpeta
        blobs = cliente_storage.list_blobs(nombre_bucket, prefix=nombre_carpeta)

        for blob in blobs:
            # Obtiene el nombre del archivo relativo a la carpeta que estamos viendo
            nombre_relativo = blob.name.replace(nombre_carpeta, '', 1)
            if nombre_relativo: # Evita añadir una entrada vacía si la carpeta misma aparece
                nombres_archivos.append(nombre_relativo)
        
        return nombres_archivos

    except Exception as e:
        print(f"Ocurrió un error al acceder al bucket: {e}")
        return []


In [18]:
obtener_nombres_de_archivos(BUCKET_NAME, VIDEO_FOLDER_PATH)

['mex_hon_000.mp4',
 'mex_hon_001.mp4',
 'mex_hon_002.mp4',
 'mex_hon_003.mp4',
 'mex_hon_004.mp4',
 'mex_hon_005.mp4',
 'mex_hon_006.mp4',
 'mex_hon_007.mp4',
 'mex_hon_008.mp4',
 'mex_hon_009.mp4',
 'mex_hon_010.mp4',
 'mex_hon_011.mp4',
 'mex_hon_012.mp4',
 'mex_hon_013.mp4',
 'mex_hon_014.mp4',
 'mex_hon_015.mp4',
 'mex_hon_016.mp4',
 'mex_hon_017.mp4',
 'mex_hon_018.mp4',
 'mex_hon_019.mp4',
 'mex_hon_020.mp4',
 'mex_hon_021.mp4',
 'mex_hon_022.mp4',
 'mex_hon_023.mp4',
 'mex_hon_024.mp4',
 'mex_hon_025.mp4',
 'mex_hon_026.mp4',
 'mex_hon_027.mp4',
 'mex_hon_028.mp4',
 'mex_hon_029.mp4',
 'mex_hon_030.mp4',
 'mex_hon_031.mp4',
 'mex_hon_032.mp4',
 'mex_hon_033.mp4',
 'mex_hon_034.mp4',
 'mex_hon_035.mp4',
 'mex_hon_036.mp4',
 'mex_hon_037.mp4',
 'mex_hon_038.mp4',
 'mex_hon_039.mp4',
 'mex_hon_040.mp4',
 'mex_hon_041.mp4',
 'mex_hon_042.mp4',
 'mex_hon_043.mp4',
 'mex_hon_044.mp4',
 'mex_hon_045.mp4',
 'mex_hon_046.mp4',
 'mex_hon_047.mp4',
 'mex_hon_048.mp4',
 'mex_hon_049.mp4',
